## Creation of ML Pipeline to monitor and choose the best model to be deployed.

In [1]:
# !pip install mlflow
# !pip install pmdarima
# # !pip install fbprophet
# !python -m pip install prophet

In [15]:
import mlflow
import pandas as pd
import pmdarima as pm
import numpy as np
# from fbprophet import Prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

## Setup of MLFLow UI

In [3]:
# Set up mlflow tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("forecasting-model")


<Experiment: artifact_location='mlflow-artifacts:/312500159318999852', creation_time=1677306715321, experiment_id='312500159318999852', last_update_time=1677306715321, lifecycle_stage='active', name='forecasting-model', tags={}>

## Splitting data into training - 17 years and test - 5 years

In [4]:
# Load data
data = pd.read_csv("Electricity.csv")
data = data.drop(['Unnamed: 0','location','stateDescription','sectorid','fueltypeid','fuelTypeDescription','total-consumption-btu-units'], axis=1)
data['period'] = pd.to_datetime(data['period'], format='%Y-%m', errors='coerce').dropna()
data.rename(columns = {'period':'ds','total-consumption-btu':'y','sectorDescription':'sector'}, inplace = True)

dict = {}
lst = list(data['sector'].unique())

for i in range(len(lst)):
    dict[i] = data[data['sector']==lst[i]]
    

In [5]:
dict[10]

,ds,sector,y
10,2022-11-01,Industrial Non-CHP,5.01918
22,2022-10-01,Industrial Non-CHP,5.19986
36,2022-09-01,Industrial Non-CHP,5.60636
45,2022-08-01,Industrial Non-CHP,5.94788
60,2022-07-01,Industrial Non-CHP,6.00998
...,...,...,...
3623,2001-05-01,Industrial Non-CHP,7.23900
3632,2001-04-01,Industrial Non-CHP,7.08400
3649,2001-03-01,Industrial Non-CHP,8.43800
3656,2001-02-01,Industrial Non-CHP,6.78400


In [20]:
# Split data into training and testing sets on example sector of Coal Consumption
split_date = '2017-12-01'

df_train = dict[0].loc[dict[0]['ds'] <= split_date]
df_test = dict[0].loc[dict[0]['ds'] > split_date]


## Track ARIMA in MLFLOW

In [7]:
# Start mlflow run for ARIMA model

with mlflow.start_run(run_name = 'PMDARIMA'):
    # Find the best ARIMA model using auto_arima
    arima_model = pm.auto_arima(df_train['y'], seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)

    # Make predictions on test set
    y_pred = arima_model.predict(n_periods=len(df_test['y']), nfreq='MS')

    # Calculate mean squared error
    mse = mean_squared_error(df_test['y'], y_pred)
    mae = mean_absolute_error(df_test['y'], y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(df_test['y'], y_pred)
    r2 = r2_score(df_test['y'], y_pred)
    variance = explained_variance_score(df_test['y'], y_pred)

    # Log model parameters and metrics to mlflow
    mlflow.log_param("model", "ARIMA")
    mlflow.log_param("test_size", 0.23)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mape", mape)
    mlflow.log_metric('r2_score',r2)
    mlflow.log_metric('explained_variance_score',variance)

    # Save model as artifact
    mlflow.pmdarima.log_model(arima_model, "model")


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.23 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3914.869, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3656.597, Time=0.06 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=2931.490, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=2932.933, Time=0.07 sec


/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value e

 ARIMA(1,0,2)(0,0,0)[0]             : AIC=2875.193, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=3453.025, Time=0.09 sec


/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,0,3)(0,0,0)[0]             : AIC=inf, Time=0.17 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=inf, Time=0.11 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=0.20 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=2837.040, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=2834.318, Time=0.07 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2862.983, Time=0.05 sec


...

/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,0,4)(0,0,0)[0] intercept   : AIC=2800.550, Time=0.10 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=inf, Time=0.31 sec


/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,0,5)(0,0,0)[0] intercept   : AIC=2811.335, Time=0.31 sec
 ARIMA(0,0,5)(0,0,0)[0] intercept   : AIC=2799.213, Time=0.19 sec
 ARIMA(0,0,5)(0,0,0)[0]             : AIC=inf, Time=0.27 sec

Best model:  ARIMA(0,0,5)(0,0,0)[0] intercept
Total fit time: 2.750 seconds


/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Track SARIMA in MLFLOW

In [8]:
# Start mlflow run
    
with mlflow.start_run(run_name = 'SARIMA'):

    # Define SARIMA model
    model = SARIMAX(df_train['y'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

    # Fit SARIMA model
    model_fit = model.fit()

    # Make predictions on test set
    y_pred = model_fit.forecast(len(df_test['y']))

    # Calculate mean squared error
    mse = mean_squared_error(df_test['y'], y_pred)
    mae = mean_absolute_error(df_test['y'], y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(df_test['y'], y_pred)
    r2 = r2_score(df_test['y'], y_pred)
    variance = explained_variance_score(df_test['y'], y_pred)

    # Log model parameters and metrics to mlflow
    mlflow.log_param("model", "SARIMA")
    mlflow.log_param("test_size", 0.23)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mape", mape)
    mlflow.log_metric('r2_score',r2)
    mlflow.log_metric('explained_variance_score',variance)

    # Save model as artifact
    mlflow.statsmodels.autolog(log_models=True, disable=False, exclusive=False, disable_for_unsupported_versions=False, silent=False, registered_model_name='SARIMA')


/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.72116D+00    |proj g|=  7.62036D-02

At iterate    5    f=  5.66691D+00    |proj g|=  7.87169D-03

At iterate   10    f=  5.66078D+00    |proj g|=  5.52259D-03

At iterate   15    f=  5.61836D+00    |proj g|=  4.03782D-02

At iterate   20    f=  5.61465D+00    |proj g|=  1.80371D-03

At iterate   25    f=  5.61416D+00    |proj g|=  9.89021D-04

At iterate   30    f=  5.59915D+00    |proj g|=  6.35998D-02

At iterate   35    f=  5.59435D+00    |proj g|=  4.62332D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function 

/Users/prajuktapradhan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


## Track Prophet in MLFLOW

In [25]:
# Start mlflow run for Prophet model

with mlflow.start_run(run_name = 'PROPHET'):
    # Prepare data for Prophet model
    prophet_data = pd.DataFrame({"ds": df_train['ds'], "y": df_train['y']})

    # Train Prophet model
    prophet_model = Prophet()
    prophet_model.fit(prophet_data)

    # Make predictions on test set
    future = prophet_model.make_future_dataframe(periods= len(df_test['y']), freq = 'MS')
    forecast = prophet_model.predict(future)["yhat"][-len(df_test['y']):]

    # Calculate mean squared error
    mse = mean_squared_error(df_test['y'], forecast)
    mae = mean_absolute_error(df_test['y'], forecast)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(df_test['y'], forecast)
    r2 = r2_score(df_test['y'], forecast)
    variance = explained_variance_score(df_test['y'], forecast)

    # Log model parameters and metrics to mlflow
    mlflow.log_param("model", "Prophet")
    mlflow.log_param("test_size", 0.23)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mape", mape)
    mlflow.log_metric('r2_score',r2)
    mlflow.log_metric('explained_variance_score',variance)
    # Save model as artifact
    mlflow.prophet.log_model(prophet_model,"model")


21:31:56 - cmdstanpy - INFO - Chain [1] start processing
21:31:56 - cmdstanpy - INFO - Chain [1] done processing
